In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93,54,58,Rear


In [3]:
test = pd.read_csv('test.csv')
test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2c9180975a056a64015a1e10d3f270fe,Right,67,0.63,107,169
1,2c9180975a056a64015a1de4deb16bdc,Front,16,0.88,61,69
2,2c9180975a056a64015a1e0e70ea70ce,Right,44,1.15,232,202
3,2c9180975a056a64015a1dfed0c46ec6,Right,50,1.10,137,125
4,2c9180975a056a64015a1dfed0c46ec7,Front,30,0.95,99,104


In [4]:
train.corr()[((train.corr() < -0.85) | (train.corr() > 0.85)) & (train.corr() != 1)]

,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
AngleOfSign,NaN,NaN,NaN,NaN
SignAspectRatio,NaN,NaN,NaN,NaN
SignWidth,NaN,NaN,NaN,0.856428
SignHeight,NaN,NaN,0.856428,NaN


In [5]:
import scipy.stats as sp

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()
train['DetectedCamera'] = le.fit_transform(train['DetectedCamera'])
test['DetectedCamera'] = le.fit_transform(test['DetectedCamera'])
train['DetectedCamera'].head()

0    2
1    2
2    0
3    2
4    2
Name: DetectedCamera, dtype: int64

In [8]:
le = LabelEncoder()
train['SignFacing (Target)'] = le.fit_transform(train['SignFacing (Target)'])
train['SignFacing (Target)'].head()

0    2
1    2
2    0
3    2
4    2
Name: SignFacing (Target), dtype: int64

In [9]:
col = ['SignWidth','SignHeight']
for i in col:
    print(sp.pearsonr(train[i],train['SignFacing (Target)']))

(-0.41232095212122594, 0.0)
(-0.45175005340893243, 0.0)


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV

In [11]:
features = ['DetectedCamera','AngleOfSign', 'SignAspectRatio','SignWidth']
X = train[features]
y = train['SignFacing (Target)']

In [24]:
param_test = {
    'max_depth':[5,10,15,20,25],
    'min_samples_split':[2,10,20,50,100]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=2,min_samples_split=2,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'max_depth': 10, 'min_samples_split': 20}, 0.96175055817339739)

In [13]:
param_test = {
    'max_depth':[15,16,17,18,19,20],
    'min_samples_split':[20,30,40,50]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=2,min_samples_split=2,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'max_depth': 17, 'min_samples_split': 30}, 0.96084111617621848)

In [14]:
param_test = {
    'min_samples_split':[20,21,22,23,24,25,26,27,28,29,30]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=17,min_samples_split=2,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'min_samples_split': 30}, 0.96084111617621848)

In [15]:
param_test = {
    'min_samples_leaf':[1,2,5,10,15]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=17,min_samples_split=30,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'min_samples_leaf': 1}, 0.96084111617621848)

In [12]:
rf = RandomForestClassifier(n_estimators=400, max_depth=9,min_samples_split=30,
                            min_samples_leaf=2,n_jobs=-1,random_state=111,oob_score=True)

In [13]:
rf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=2, min_samples_split=30,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
            oob_score=True, random_state=111, verbose=0, warm_start=False)

In [14]:
prob = rf.predict_proba(test[features])
submission = pd.DataFrame(data=prob,index = test.Id,columns=['Front','Left','Rear','Right'])
submission.to_csv('rfr.csv')
submission.head()

,Front,Left,Rear,Right
Id,,,,
2c9180975a056a64015a1e10d3f270fe,0.996912,0.000038,0.001223,0.001827
2c9180975a056a64015a1de4deb16bdc,0.998235,0.000426,0.000122,0.001216
2c9180975a056a64015a1e0e70ea70ce,0.996585,0.000119,0.001592,0.001705
2c9180975a056a64015a1dfed0c46ec6,0.997007,0.000083,0.001218,0.001691
2c9180975a056a64015a1dfed0c46ec7,0.998866,0.000391,0.000089,0.000654
